<a href="https://colab.research.google.com/github/badrinarayanan02/LLM/blob/main/2348507_LLMLab4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Finetuning BERT LLM for Sentiment Analysis


Going to Finetune DistillBERT llm model using Transformers and Tensforflow



DistillBERT - It is a smaller version of BERT. It tries to mimic the pretrained model BERT. It has less computations.

Loading the Dataset

In [57]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizerFast
import tensorflow as tf
import numpy as np
from transformers import TFDistilBertForSequenceClassification, Trainer, TFTrainingArguments

In [ ]:
df=messages = pd.read_csv('/content/SMSSpamCollection..txt', sep='\t',
                           names=["label", "message"])
df.head()


,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
df.shape

(5572, 2)

Converting into Independent and Dependent Features

In [ ]:
X = list(df['message'])

In [ ]:
y = list(df['label'])

In [ ]:
X[:5]

['Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...',
 'Ok lar... Joking wif u oni...',
 "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's",
 'U dun say so early hor... U c already then say...',
 "Nah I don't think he goes to usf, he lives around here though"]

In [ ]:
y[:5]

['ham', 'ham', 'spam', 'ham', 'ham']

In [ ]:
y_series = pd.Series(y)

Converting the dependent feature to a numerical value (boolean)

In [ ]:
dummies = pd.get_dummies(y_series,drop_first=True)

In [ ]:
dummies

,spam
0,False
1,False
2,True
3,False
4,False
...,...
5567,True
5568,False
5569,False
5570,False


Converting into a list

In [ ]:
y = list(pd.get_dummies(dummies,drop_first=True)['spam'])

In [ ]:
y[:5]

[False, False, True, False, False]

Splitting the Data

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [ ]:
X_train[:10]

['Reply to win £100 weekly! Where will the 2006 FIFA World Cup be held? Send STOP to 87239 to end service',
 'Hello. Sort of out in town already. That . So dont rush home, I am eating nachos. Will let you know eta.',
 'How come guoyang go n tell her? Then u told her?',
 'Hey sathya till now we dint meet not even a single time then how can i saw the situation sathya.',
 'Orange brings you ringtones from all time Chart Heroes, with a free hit each week! Go to Ringtones & Pics on wap. To stop receiving these tips reply STOP.',
 'Sitting in mu waiting for everyone to get out of my suite so I can take a shower',
 'Finish liao... U?',
 'URGENT! Your Mobile No 07808726822 was awarded a £2,000 Bonus Caller Prize on 02/09/03! This is our 2nd attempt to contact YOU! Call 0871-872-9758 BOX95QU',
 'Probably not, still going over some stuff here',
 'Wah lucky man... Then can save money... Hee...']

Installing Transformers

In [ ]:
pip install transformers

After Installing Transformers

1) Call the pretrained model

2) Call the tokenizer (For specific model specific tokenizer is there)

3) Convert the encodings to a data object

Loading the pretrained model

In [ ]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Performing Encodings

In [ ]:
train_encodings = tokenizer(X_train, truncation = True, padding = True)
test_encodings = tokenizer(X_test, truncation = True, padding=True)

In [ ]:
train_encodings[:10]

Converting these encodings into a dataset objects using tensorflow

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train
)).batch(16)

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    y_test
)).batch(16)

In [ ]:
train_dataset

<_BatchDataset element_spec=({'input_ids': TensorSpec(shape=(None, 238), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(None, 238), dtype=tf.int32, name=None)}, TensorSpec(shape=(None,), dtype=tf.bool, name=None))>

This is the format that is required for sequence classifications.

Initializing the Pretrained Model

In [ ]:
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [ ]:
model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5),
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics = ['accuracy']
)

Used keras because of TFTrainer deprecation in Tensorflow.

In [ ]:
history = model.fit(train_dataset, validation_data = test_dataset, epochs = 2)

Epoch 1/2
279/279 [==============================] - 141s 390ms/step - loss: 0.0661 - accuracy: 0.9803 - val_loss: 0.0295 - val_accuracy: 0.9928
Epoch 2/2
279/279 [==============================] - 108s 386ms/step - loss: 0.0238 - accuracy: 0.9942 - val_loss: 0.0234 - val_accuracy: 0.9937


In [ ]:
loss, accuracy = model.evaluate(test_dataset)
print(f"Test Accuracy: {accuracy:.4f}")

70/70 [==============================] - 6s 86ms/step - loss: 0.0234 - accuracy: 0.9937
Test Accuracy: 0.9937


In [ ]:
predictions = model.predict(test_dataset)

70/70 [==============================] - 6s 84ms/step


In [ ]:
predictions

TFSequenceClassifierOutput(loss=None, logits=array([[ 2.856457 , -2.7873452],
       [ 2.9233966, -2.8715656],
       [ 2.9492729, -2.8643558],
       ...,
       [ 2.9760509, -2.9321158],
       [ 2.9538133, -2.8667595],
       [ 2.940574 , -2.8756895]], dtype=float32), hidden_states=None, attentions=None)

These are logits score. Logits - Raw scores output by the model.

Conversion of Logits to Probabilities

In [ ]:
logits = predictions.logits
probabilities = tf.nn.softmax(logits,axis=1)
predicted_labels = tf.argmax(probabilities, axis=-1).numpy()

In [ ]:
y_test_np = np.array(y_test)
print('Predicted Labels:',predicted_labels[:20])
print("True Labels:", y_test_np[:20])

Predicted Labels: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
True Labels: [False False False False False False False False False False False False
 False False  True False False False False False]


In [ ]:
model.save_pretrained("sentiment_analysis_model")

Predictions on user inputs

In [ ]:
def preprocess_input(text,tokenizer):
  encodings = tokenizer(text, truncation = True, padding = True, return_tensors='tf')
  return encodings

In [44]:
def predict_sentiment(text,model,tokenizer):
  input_encodings = preprocess_input(text,tokenizer)
  predictions = model(input_encodings)
  logits = predictions.logits
  probabilities = tf.nn.softmax(logits,axis=1)
  predicted_labels = tf.argmax(probabilities, axis=-1).numpy()[0]

  label_map = {0:'ham',1:'spam'}
  predictedclass = label_map[predicted_labels]
  return predictedclass

In [64]:
input = "Congratulations! You won a $1,000 gift card. Click the link to claim your prize now: [link]"
predicted_class = predict_sentiment(input,model,tokenizer)
print(f"Message is Classified as {predicted_class}")

Message is Classified as spam


# Conclusion

Finetuned the DistilBert LLM model. Gave a custom dataset that includes spam or ham. The model will analyse the sentiment of the message whether it is ham or spam. Used transformers to do all the required operations. Tensorflow is used for the conversion of encodings to dataset object. Used keras because I faced an issue with TFTrainer in tensorflow, since it is deprecated.